In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost
import math
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn import model_selection, tree, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from xgboost import XGBRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

In [15]:
data_listing= pd.read_csv('/Users/tjmask/Desktop/data competition/listings_details_nice.csv', delimiter=",")


In [17]:
data_listing = data_listing.drop(columns=["Unnamed: 0", 'scrape_id',"latitude",'longitude',"host_id"])
data_listing.head()

,id,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,is_location_exact,accommodates,bathrooms,bedrooms,...,calculated_host_listings_count_shared_rooms,reviews_per_month,host_years,zip_has,host_response_price,neighbourhood_cleansed_price,property_type_price,room_type_price,bed_type_price,cancellation_policy_price
0,6005903,95.339076,0,1,1,1,1,2,1.0,1,...,0,2.007858,7.01,5477,163.951228,302.589491,199.172745,243.827773,104.483092,142.190586
1,6005903,95.339076,0,1,1,1,1,2,1.0,1,...,0,2.007858,7.01,5477,163.951228,302.589491,199.172745,243.827773,104.483092,142.190586
2,6005903,95.339076,0,1,1,1,1,2,1.0,1,...,0,2.007858,7.01,5477,163.951228,302.589491,199.172745,243.827773,104.483092,142.190586
3,6005903,95.339076,0,1,1,1,1,2,1.0,1,...,0,2.007858,7.01,5477,163.951228,302.589491,199.172745,243.827773,104.483092,142.190586
4,6005903,95.339076,0,1,1,1,1,2,1.0,1,...,0,2.007858,7.01,5477,163.951228,302.589491,199.172745,243.827773,104.483092,142.190586


In [35]:
data_calendar = pd.read_csv('/Users/tjmask/Desktop/data competition/calendar_sample.csv', delimiter=",")

In [36]:
data_calendar.head()

,Unnamed: 0,listing_id,date,avg_adjusted_price,weekday,months,month_day,is_weekday,is_travel_month,is_holiday
0,1,8789,2019-02-09,91.0,Saturday,2,02-09,1,0,0
1,2,8789,2019-02-10,87.0,Sunday,2,02-10,0,0,0
2,3,8789,2019-02-11,88.0,Monday,2,02-11,0,0,0
3,4,8789,2019-02-12,87.0,Tuesday,2,02-12,0,0,0
4,5,8789,2019-02-13,86.0,Wednesday,2,02-13,0,0,0


In [37]:
df_merge = pd.merge(data_calendar, data_listing, how='inner', left_on=data_calendar['listing_id'], right_on=data_listing['id'])
df_merge.eval('time_neighbor = is_weekday*neighbourhood_cleansed_price' , inplace=True)

In [38]:
df_merge = df_merge.drop(columns=["Unnamed: 0", "key_0", "date"])

In [39]:
df_merge.head()

,listing_id,avg_adjusted_price,weekday,months,month_day,is_weekday,is_travel_month,is_holiday,id,host_response_rate,...,reviews_per_month,host_years,zip_has,host_response_price,neighbourhood_cleansed_price,property_type_price,room_type_price,bed_type_price,cancellation_policy_price,time_neighbor
0,8789,91.0,Saturday,2,02-09,1,0,0,8789,95.33908,...,0.41,10.28,960,163.9512,255.4549,199.1727,243.8278,188.0258,217.8662,255.4549
1,8789,91.0,Saturday,2,02-09,1,0,0,8789,90.00000,...,0.39,10.28,960,220.9754,255.4549,199.1727,243.8278,188.0258,217.8662,255.4549
2,8789,91.0,Saturday,2,02-09,1,0,0,8789,93.00000,...,0.40,10.28,960,220.9754,255.4549,199.1727,243.8278,188.0258,217.8662,255.4549
3,8789,91.0,Saturday,2,02-09,1,0,0,8789,100.00000,...,0.37,10.28,960,220.9754,255.4549,199.1727,243.8278,188.0258,217.8662,255.4549
4,8789,91.0,Saturday,2,02-09,1,0,0,8789,93.00000,...,0.38,10.28,960,220.9754,255.4549,199.1727,243.8278,188.0258,217.8662,255.4549


In [58]:
df_random = df_merge.sample(n=180000,replace=False, random_state=12)

In [138]:
features = df_random[['is_weekday','is_travel_month','time_neighbor',
       'is_holiday', 'zip_has', 'host_response_price',
       'neighbourhood_cleansed_price', 'property_type_price',
       'room_type_price', 'bed_type_price', 'cancellation_policy_price','host_years',
                      
        'host_response_rate', 'host_is_superhost','host_has_profile_pic', 'host_identity_verified',
                      
       'is_location_exact', 'bathrooms', 'bedrooms',
                      
       'cleaning_fee', 'guests_included',
       'extra_people', 'minimum_nights',
                      
       'availability_60','availability_365',
                      
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value','reviews_per_month',
                      
       'instant_bookable', 'require_guest_profile_picture',
        
       'require_guest_phone_verification']]

In [139]:
prices = pd.DataFrame(df_random["avg_adjusted_price"])

In [140]:
np.shape(features)

(180000, 35)

In [141]:
regr = linear_model.LinearRegression()
X_train, X_test, y_train, y_test = model_selection.train_test_split(features, prices ,test_size=0.3)

In [142]:
regr.fit(X_train, y_train)
print(regr.predict(X_test))

[[228.50421308]
 [118.82878736]
 [347.26255299]
 ...
 [321.13095414]
 [ 99.40248947]
 [331.30182152]]


In [143]:
regr.score(X_test,y_test)

0.3367837400408449

In [147]:
## using xgboost mo
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

train_X, test_X, train_y, test_y = train_test_split(features, prices, test_size=0.25)

my_imputer = Imputer()
train_X = my_imputer.fit_transform(train_X)
test_X = my_imputer.transform(test_X)

//anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [148]:
from xgboost import XGBRegressor
my_model = XGBRegressor(learning_rate=0.1, n_estimators=550, max_depth=4, min_child_weight=5, seed=0,
                             subsample=0.7, colsample_bytree=0.7, gamma=0.1, reg_alpha=1, reg_lambda=1)
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(train_X, train_y, verbose=False)

[17:51:00] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0.1,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=4, min_child_weight=5, missing=None, n_estimators=550,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=1, reg_lambda=1, scale_pos_weight=1, seed=0, silent=None,
             subsample=0.7, verbosity=1)

In [149]:
## the explained_variance_score of test dataset
predictions = my_model.predict(test_X)
print(explained_variance_score(predictions,test_y))

0.8445081914640743


In [150]:
## the explained_variance_score of train dataset
predictions = my_model.predict(train_X)
print(explained_variance_score(predictions,train_y))

0.8774047833438765
